In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l
from sklearn.preprocessing import StandardScaler
import copy
import random
from tqdm import tqdm

In [ ]:
loss = nn.MSELoss()


class MLP(nn.Module):
    def __init__(self, in_features=12, hidden=512):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(1, hidden),
            nn.ReLU()
        )
        self.conv1 = nn.Sequential(nn.Conv1d(in_channels=hidden, out_channels=2048, kernel_size=1), nn.ReLU())
        self.conv2 = nn.Conv1d(in_channels=2048, out_channels=hidden, kernel_size=1)
        self.lnorm = nn.LayerNorm(hidden)
        self.fc = nn.Linear(hidden * in_features, 1)

    def forward(self, data):
        shape = data.shape
        data = data.unsqueeze(-1)
        data = self.net(data)
        out = self.conv1(data.transpose(1, 2))
        out = self.conv2(out).transpose(1, 2)
        out = self.lnorm(out + data)
        out = out.squeeze(-1).reshape(shape[0], 12 * 512)
        out = self.fc(out)
        return out


def get_net(in_features=12):
    net = MLP(in_features)
    return net

In [ ]:
def mae(net, features, labels):
    # 为了在取对数时进一步稳定该值，将小于1的值设置为1
    preds = net(features)
    mae = torch.tensor(abs(labels - preds), dtype=torch.float32)
    return mae.mean().item()

In [ ]:
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    best_mae = float('inf')
    best_weights = None
    train_ls, test_ls = [], []
    train_iter = d2l.load_array((train_features, train_labels), batch_size)
    # 这里使用的是Adam优化算法
    optimizer = torch.optim.Adam(net.parameters(),
                                 lr = learning_rate,
                                 weight_decay = weight_decay)
    for epoch in range(num_epochs):
        for X, y in train_iter:
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
        train_ls.append(mae(net, train_features, train_labels))
        if test_labels is not None:
            net.eval()
            with torch.no_grad():
                test_mae = mae(net, test_features, test_labels)
                test_ls.append(test_mae)
                if test_mae < best_mae:
                    best_mae = test_mae
                    best_weights = copy.deepcopy(net.state_dict())
            net.train()
        if best_mae < 10:
            print("best_mae:", best_mae)
    return train_ls, test_ls, best_weights

In [ ]:
def train_and_pred(train_features, test_features, train_labels, test_labels,
                   num_epochs, lr, weight_decay, batch_size):
    net = get_net()
    net = net.to(device)
    train_ls, test_ls, best_weights = train(net, train_features, train_labels, test_features, test_labels,
                        num_epochs, lr, weight_decay, batch_size)
#     d2l.plot(np.arange(1, num_epochs + 1), [train_ls, test_ls], xlabel='epoch',
#              ylabel='mse', xlim=[1, num_epochs], legend=['train', 'valid'])
#     print(f'训练mse：{float(min(train_ls)):f}')
#     print(f'测试mse：{float(min(test_ls)):f}')
    # 将网络应用于测试集。
    net.eval()
    with torch.no_grad():
        net.load_state_dict(best_weights)
        preds = net(test_features).cpu().detach().numpy()
    net.train()

    return preds

In [ ]:
def evaluate_metrics(y_test_list, y_pred_list):
    mae, rmse = [], []
    for y_test, y_pred in zip(y_test_list, y_pred_list):
        mae.append(np.mean(np.abs(y_test - y_pred)))
        rmse.append(np.sqrt(np.mean((y_test - y_pred) ** 2)))
        
    mae_mean, mae_std = np.mean(mae), np.std(mae)
    rmse_mean, rmse_std = np.mean(rmse), np.std(rmse)
    
    return mae_mean.item(), mae_std.item(), rmse_mean.item(), rmse_std.item()

In [ ]:
def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
k, weight_decay = 5, 0
num_epochs, lr, batch_size = 100, 1e-4, 8

train_data = pd.read_csv("../dataset/CME/train_val_set.csv", header=None)
test_data = pd.read_csv("../dataset/CME/test_set.csv", header=None)

scaler = StandardScaler()
X_train = scaler.fit_transform(train_data.iloc[:, 2:-1].values)
X_test = scaler.transform(test_data.iloc[:, 2:-1].values)

train_labels = train_data.iloc[:, -1].values.reshape(-1, 1)
test_labels = test_data.iloc[:, -1].values.reshape(-1, 1)

train_features = torch.tensor(X_train, dtype=torch.float32).to(device)
test_features = torch.tensor(X_test, dtype=torch.float32).to(device)
train_labels = torch.tensor(train_labels, dtype=torch.float32).to(device)
test_labels = torch.tensor(test_labels, dtype=torch.float32).to(device)

set_random_seed(42)


y_pred = train_and_pred(train_features, test_features, train_labels, test_labels, num_epochs, 
                        lr, weight_decay, batch_size)

test_labels = test_labels.cpu().detach().numpy()
    

print("MAE:", np.mean(np.abs(y_pred - test_labels)).item())
print("RMSE:", np.sqrt(np.mean((y_pred - test_labels) ** 2)))